In [78]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Model Evaluation Metrics
from sklearn.metrics import classification_report, accuracy_score

#Model Selection
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model  import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [126]:
df_train = pd.read_csv('./Data/train.csv')
df_test = pd.read_csv('./Data/test.csv')
df_gender = pd.read_csv('./Data/gender_submission.csv')

In [80]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [81]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


# Eksik Veri Doldurma İşlemi

## Train Setinde Eksik Veri Doldurma İşlemi

In [82]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Age ve Embarked özelliklerindeki eksik değerleri dolduracağız fakat cabin değişkenini çok fazla eksik değerinin
olması ve dolu değerlerinin de bir kısmının anlamsız veriler içermesi sebebiyle atacağız

In [83]:
df_train['Age'].mean()

29.69911764705882

In [84]:
#Yaş değişkeninin eksik değerlerini ortalama yaş değeri ile dolduruyoruz
df_train['Age'].fillna(df_train['Age'].mean(),inplace=True)

In [85]:
df_train.Age.isnull().sum()

0

In [86]:
df_train['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [87]:
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0],inplace=True)

In [88]:
df_train.Embarked.isnull().sum()

0

## Test Setinde Eksik Veri Doldurma İşlemi

In [89]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Age ve Fare Özelliklerindeki eksik değerleri dolduracağız fakat cabin değişkenini çok fazla eksik değerinin
olması ve dolu değerlerinin de bir kısmının anlamsız veriler içermesi sebebiyle atacağız

In [90]:
df_test['Age'].mean()

30.272590361445783

In [91]:
#Yaş değişkeninin eksik değerlerini ortalama yaş değeri ile dolduruyoruz
df_test['Age'].fillna(df_test['Age'].mean(),inplace=True)

In [92]:
df_test.Age.isnull().sum()

0

In [93]:
df_test['Fare'].mean()

35.627188489208635

In [94]:
df_test['Fare'].fillna(df_test['Fare'].mean(),inplace=True)

In [95]:
df_test.Fare.isnull().sum()

0

# Aykırı Gözlemlerin Temizlenmesi

## Train Setinde Aykırı Gözlemlerin Temizlenmesi

In [96]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.000000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.000000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.000000,3.0,3.0000
Age,891.0,29.699118,13.002015,0.42,22.0000,29.699118,35.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.000000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.000000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.454200,31.0,512.3292


### Fare İçin Aykırı Gözlem Kontrolü ve Temizliği

In [97]:
#Fare için IQR değeri hesaplanıyor
IQR = df_train.Fare.quantile(0.75) - df_train.Fare.quantile(0.25)
IQR

23.0896

In [98]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_train['Fare'].quantile(0.25)-(IQR*3)
üst_sınır=df_train['Fare'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-61.358399999999996, 100.2688)

In [99]:
# Fare için Alt sınırın altında gözlem yok
df_train[df_train['Fare']<alt_sınır]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [100]:
#Fare değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_train[df_train['Fare']>üst_sınır].shape

(53, 12)

In [101]:
# Üst sınırın üstünde olan fare değişkenlerini üst sınıra baskılıyoruz
df_train.loc[df_train['Fare']>üst_sınır,'Fare'] = üst_sınır

In [102]:
# Fare değişkeni için aykırı gözlemler baskılanmış oldu
df_train[df_train['Fare']>üst_sınır].shape

(0, 12)

### Age İçin Aykırı Gözlem Kontrolü ve Temizliği

In [103]:
#Age için IQR değeri hesaplanıyor
IQR = df_train.Age.quantile(0.75) - df_train.Age.quantile(0.25)
IQR

13.0

In [104]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_train['Age'].quantile(0.25)-(IQR*3)
üst_sınır=df_train['Age'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-17.0, 74.0)

In [105]:
# Age için Alt sınırın altında gözlem yok
df_train[df_train['Age']<alt_sınır]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [106]:
#Age değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_train[df_train['Age']>üst_sınır].shape

(1, 12)

In [107]:
# Üst sınırın üstünde olan Age değişkenlerini üst sınıra baskılıyoruz
df_train.loc[df_train['Age']>üst_sınır,'Age'] = üst_sınır

In [108]:
# Age değişkeni için aykırı gözlemler baskılanmış oldu
df_train[df_train['Age']>üst_sınır].shape

(0, 12)

## Test Setinde Aykırı Gözlemlerin Temizlenmesi

### Fare İçin Aykırı Gözlem Kontrolü ve Temizliği

In [109]:
#Fare için IQR değeri hesaplanıyor
IQR = df_test.Fare.quantile(0.75) - df_test.Fare.quantile(0.25)
IQR

23.6042

In [110]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_test['Fare'].quantile(0.25)-(IQR*3)
üst_sınır=df_test['Fare'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-62.9168, 102.3126)

In [111]:
# Fare için Alt sınırın altında gözlem yok
df_test[df_test['Fare']<alt_sınır]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [112]:
#Fare değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_test[df_test['Fare']>üst_sınır].shape

(31, 11)

In [113]:
# Üst sınırın üstünde olan fare değişkenlerini üst sınıra baskılıyoruz
df_test.loc[df_test['Fare']>üst_sınır,'Fare'] = üst_sınır

In [114]:
# Fare değişkeni için aykırı gözlemler baskılanmış oldu
df_test[df_test['Fare']>üst_sınır].shape

(0, 11)

### Age İçin Aykırı Gözlem Kontrolü ve Temizliği

In [115]:
#Age için IQR değeri hesaplanıyor
IQR = df_test.Age.quantile(0.75) - df_test.Age.quantile(0.25)
IQR

12.75

In [116]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_test['Age'].quantile(0.25)-(IQR*3)
üst_sınır=df_test['Age'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-15.25, 74.0)

In [117]:
# Age için Alt sınırın altında gözlem yok
df_test[df_test['Age']<alt_sınır]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [118]:
#Age değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_test[df_test['Age']>üst_sınır].shape

(1, 11)

In [119]:
# Üst sınırın üstünde olan Age değişkenlerini üst sınıra baskılıyoruz
df_test.loc[df_test['Age']>üst_sınır,'Age'] = üst_sınır

In [120]:
# Age değişkeni için aykırı gözlemler baskılanmış oldu
df_test[df_test['Age']>üst_sınır].shape

(0, 11)

# Gereksiz Özelliklerin Atılması

In [121]:
df_train.drop(columns=['PassengerId','Cabin','Ticket','Name'],inplace=True)
df_test.drop(columns=['PassengerId','Cabin','Ticket','Name'],inplace=True)

In [122]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [125]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,102.3126,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [128]:
# train_test_split fonksiyonunun ayırdığı gibi verimizi bağımlı değişken ve bağımsız değişkenlerine ayırıyoruz.
X_train = df_train.drop(columns=['Survived'])
y_train = df_train['Survived']
X_test = df_test
y_test = df_gender

Buradan sonra
- Kategorik değişkenler sayısal değişkenlere dönüştürülecek
- GridSearchCV kullanılarak LogisticRegression ve KNeighborsClassifier'ın classification_report sonuçlarına bakılacak.